Вы работаете аналитиком данных в онлайн-кинотеатре *SkyCinema*.\
Сегодня вы будете работать с **ивентами** (event - *событие*). Ивенты - это записи о клиентских событиях, таких как включение плеера с видео или заход на сайт онлайн-кинотеатра. Одной из важнейших задач аналитики является изучение и правка ошибочных ивентов.

Ваша задача - изучить датасет на наличие ошибок.

Импортируйте файл *event_list.xlsx* и сохраните его в датафрейм.

**Описание данных**

    - account_id - идентификатор пользователя
    - server_time - время совершения действия
    - action_id - идентификатор (тег) действия
    - device_type - тип устройства
    - user_browser - браузер
    - screen_type - тип страницы, на которой совершается действие.
*В рамках данного технического задания вам необходимо взять только те строки, у которых колонка screen_type принимает значение "player".*
    
**Описание ивентов**

    - 'screenview' - заход на страницу с проигрывателем тайтла
    - 'click' - клик по плееру
    - 'playback_start' - включение проигрывателя
    - 'playback_pause' - постановка проигрывателя на паузу
    - 'playback_unpause' - снятия проигрывателя с паузы
    - 'playback_stop'  - выключение страницы с проигрывателем

In [1]:
import pandas as pd
# Загрузка данных
events = pd.read_excel('event_list.xlsx')
events = events[events['screen_type'] == 'player']  # ФИЛЬТР - только те строки, у которых колонка screen_type принимает значение "player".
events.head()
# events.dtypes

,account_id,server_time,screen_type,action_id,device_type,user_browser
33,102313,2021-10-01 09:24:12,player,playback_start,desktop_web,Chrome
34,102313,2021-10-01 09:24:13,player,playback_pause,desktop_web,Chrome
43,102313,2021-10-01 11:53:54,player,playback_start,desktop_web,Chrome
44,102313,2021-10-01 11:53:56,player,playback_pause,desktop_web,Chrome
45,102313,2021-10-01 11:53:56,player,click,desktop_web,Chrome


### Задача 1

Выделите 10 самых активных пользователей (тех, что сделали больше всего действий за данный период времени).\
Каким браузером чаще всего пользовались эти 10 пользователей?\
Какое действие занимает наибольший процент среди всех действий этих 10 пользователей?

In [2]:
active10 = events.groupby(['account_id'])\
.agg(  cnt_actions = ('server_time','count') )\
.reset_index().sort_values('cnt_actions', ascending = False)\
.head(10)['account_id']
print(active10) # Получили список 10 самых активных.
# type(active10) # Тип серия, т.к скобка одна.

36     102598
145    103398
37     102605
76     102904
108    103149
199    103804
120    103219
10     102405
74     102883
138    103364
Name: account_id, dtype: int64


In [3]:
# Фильтруем исх таблицу - только 10 самых активных, столбцы оставляем все.
active10_full = events[events['account_id'].isin(active10)] 
# print(active10_full)

In [5]:
# Каким браузером чаще всего пользовались эти 10 пользователей:
active10_full.groupby(['user_browser'])\
.agg(  cnt_actions = ('server_time','count') )\
.reset_index().sort_values('cnt_actions', ascending = False)\
.head(60)

,user_browser,cnt_actions
4,Yandex,1004
0,Chrome,968
2,Safari,301
3,Samsung Browser,141
1,Edge,128


In [6]:
# Какое действие занимает наибольший процент среди всех действий этих 10 пользователей:
top_actions = active10_full.groupby(['action_id'])\
.agg(  cnt_actions = ('server_time','count') )\
.reset_index().sort_values('cnt_actions', ascending = False)\
.head(60)

total_actions = top_actions['cnt_actions'].sum()  # Считаем сумму всех действий
# print(total_actions)

top_actions['action_share_%'] = 100*top_actions['cnt_actions'] / total_actions  # Считаем процентную долю каждого действия.
print(top_actions)

          action_id  cnt_actions  action_share_%
0             click         1595       62.745869
1    playback_pause          334       13.139260
4  playback_unpause          313       12.313139
2    playback_start          181        7.120378
3     playback_stop          119        4.681353


### Задача 2

Проверьте данные на состоятельность:

1. Есть ли клиенты, которые снимали видео с паузы хотя бы раз, но при этом не ставили видео на паузу ни разу?
2. Есть ли клиенты, которые ставили видео на паузу хотя бы раз, но при этом не включали проигрыватель?

С каких устройств и браузеров заходят пользователи, у которых встречаются подобные аномалии?

*Подсказка*\
Воспользуйтесь методом серии unique(), чтобы для каждого действия (ивента) создать списки с уникальными пользователями, которые совершали эти действия. \
Посмотрите на соответствующие исключения списков друг из друга, чтобы определить пользователей с проблемной последовательностью событий.

In [8]:
# 2.1 клиенты, которые снимали видео с паузы хотя бы раз, но при этом не ставили видео на паузу ни разу:
paused_users = events[events['action_id'] == 'playback_pause']['account_id'].unique()
unpaused_users = events[events['action_id'] == 'playback_unpause']['account_id'].unique()
task2_1 = set(unpaused_users) - set(paused_users)  # Переводим списки отобранных людей в МНОЖЕСТВА - set, где только уникальные, и найдем разницу.  
print('1)Клиенты, которые снимали видео с паузы хотя бы раз, но при этом не ставили видео на паузу ни разу:', task2_1)
# 2.1 С каких устройств и браузеров заходят:
filtered_task2_1 = events[events['account_id'].isin(task2_1)]
agg_task2_1 = filtered_task2_1.groupby(['user_browser', 'device_type'])\
.agg(  cnt_actions = ('server_time','count') )\
.reset_index().sort_values('cnt_actions', ascending = False)\
.head(60)
print(agg_task2_1)

# 2.2 клиенты, которые ставили видео на паузу хотя бы раз, но при этом не включали проигрыватель:
started_users = events[events['action_id'] == 'playback_start']['account_id'].unique()
task2_2 = set(paused_users) - set(started_users)
print('2)Клиенты, которые ставили видео на паузу хотя бы раз, но при этом не включали проигрыватель:', task2_2)
# 2.2 С каких устройств и браузеров заходят:
filtered_task2_2 = events[events['account_id'].isin(task2_2)]
agg_task2_2 = filtered_task2_2.groupby(['user_browser', 'device_type'])\
.agg(  cnt_actions = ('server_time','count') )\
.reset_index().sort_values('cnt_actions', ascending = False)\
.head(60)
print(agg_task2_2)

1)Клиенты, которые снимали видео с паузы хотя бы раз, но при этом не ставили видео на паузу ни разу: {102345, 103050, 102918}
  user_browser  device_type  cnt_actions
0       Chrome  desktop_web           32
1       Safari  desktop_web            4
2)Клиенты, которые ставили видео на паузу хотя бы раз, но при этом не включали проигрыватель: {103052, 102669}
  user_browser  device_type  cnt_actions
0       Safari  desktop_web            8


### Задача 3

1. Изучите пользователей, у которых есть только одно событие. Какой вид события встречается чаще всего? Какие из встречающихся событий вы бы посчитали ошибочными?
2. Изучите пользователей, у которых есть только два события. Какие из их последовательностей событий вы бы посчитали ошибочными?

In [9]:
events_asc = events.groupby(['account_id'])\
.agg(  cnt_actions = ('server_time','count') )\
.reset_index().sort_values('cnt_actions', ascending = True)\
.head(30)
# print(events_asc) # Получили список с малым числом действий.

events1 = events_asc[events_asc['cnt_actions'] == 1]['account_id'] # Отфильтруем только с 1 действием.
print('3.1/1)15 Пользователей, у которых есть только одно событие: ',events1)

task3_1 = events[events['account_id'].isin(events1)] # Из общей исх таблицы отфильтруем и возьмём только тех 15 пользователей с 1 действием.
# print(task3_1)

# Какой вид события встречается чаще всего?
agg_task3_1 = task3_1.groupby(['action_id'])\
.agg(  cnt = ('server_time','count') )\
.reset_index().sort_values('cnt', ascending = False)\
.head(20)
print('3.1/2)Какой вид события встречается чаще всего:', agg_task3_1)
print('3.1/3)Какие из встречающихся событий вы бы посчитали ошибочными?\
   ОТВЕТ:  Следующие события выглядят ошибочными, поскольку им должен предшествовать просмотр страницы:\
  click (клик по плееру) и  playback_stop  (выключение страницы с проигрывателем).')

3.1/1)15 Пользователей, у которых есть только одно событие:  70     102839
106    103140
73     102865
200    103813
18     102461
72     102853
26     102520
Name: account_id, dtype: int64
3.1/2)Какой вид события встречается чаще всего:         action_id  cnt
2   playback_stop    3
0           click    2
1  playback_start    2
3.1/3)Какие из встречающихся событий вы бы посчитали ошибочными?   ОТВЕТ:  Следующие события выглядят ошибочными, поскольку им должен предшествовать просмотр страницы:  click (клик по плееру) и  playback_stop  (выключение страницы с проигрывателем).


In [10]:
events_asc = events.groupby(['account_id'])\
.agg(  cnt_actions = ('server_time','count') )\
.reset_index().sort_values('cnt_actions', ascending = True)\
.head(30)
# print(events_asc) # Получили список с малым числом действий.

events2 = events_asc[events_asc['cnt_actions'] == 2]['account_id'] # Отфильтруем только с 2 действиями.
print('3.2/1)9 Пользователей, у которых есть только 2 события: ',events2)

task3_2 = events[events['account_id'].isin(events2)] # Из общей исх таблицы отфильтруем и возьмём только те 9 пользователей с 2 действиями.
# print(task3_2)

# Группируем и по пользователю и по событию:
agg_task3_2 = task3_2.groupby(['account_id', 'action_id', 'server_time'])\
.agg(  cnt = ('server_time','count') )\
.reset_index().sort_values('account_id', ascending = False)\
.head(40)
print('3.2/2)Группируем и по пользователю и по событию:', agg_task3_2)
print('3.2/3) Какие из их последовательностей событий вы бы посчитали ошибочными? \
   ОТВЕТ:  Ошибочными выглядят: 1) id 102682 стоп 2 раза при отсутствии просмотра и старта; \
2) id 103052 пара клик/пауза - без просмотра и старта. ')

3.2/1)9 Пользователей, у которых есть только 2 события:  186    103724
95     103052
54     102682
167    103566
28     102537
80     102943
150    103446
Name: account_id, dtype: int64
3.2/2)Группируем и по пользователю и по событию:     account_id       action_id         server_time  cnt
12      103724  playback_start 2021-10-01 13:24:46    1
13      103724   playback_stop 2021-10-01 13:24:52    1
10      103566           click 2021-10-01 10:12:56    1
11      103566  playback_start 2021-10-01 10:12:48    1
8       103446  playback_start 2021-10-01 18:43:42    1
9       103446   playback_stop 2021-10-01 18:44:14    1
6       103052           click 2021-10-01 00:06:40    1
7       103052  playback_pause 2021-10-01 00:06:53    1
4       102943  playback_pause 2021-10-01 19:48:12    1
5       102943  playback_start 2021-10-01 19:48:09    1
2       102682   playback_stop 2021-10-01 00:06:08    1
3       102682   playback_stop 2021-10-01 00:10:12    1
0       102537  playback_start 2021-1